#  **Install The Library And Load Data From the Cloud**

In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
# Mount Google Drive
drive.mount('/content/drive')
# Load the dataset from Google Drive
df_ratings = pd.read_csv('/content/drive/My Drive/rating.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
len(df_ratings)

12419285

In [ ]:
rating = pd.read_csv('/content/drive/My Drive/rating.csv', usecols=['userId','movieId','rating'], dtype={'userId': 'Int32', 'movieId': 'int32', 'rating': 'float32'})
rating.head()


,userId,movieId,rating
0,1,2,3.5
1,1,29,3.5
2,1,32,3.5
3,1,47,3.5
4,1,50,3.5


In [ ]:
movies =  pd.read_csv('movie.csv',usecols=['movieId','title'],dtype={'movieId': 'int32', 'title': 'str'})
movies

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)
...,...,...
27273,131254,Kein Bund für's Leben (2007)
27274,131256,"Feuer, Eis & Dosenbier (2002)"
27275,131258,The Pirates (2014)
27276,131260,Rentun Ruusu (2001)


## Data merging

In [ ]:
all_in_one = pd.merge(rating,movies,on='movieId')
all_in_one.head()

,userId,movieId,rating,title
0,1,2,3.5,Jumanji (1995)
1,5,2,3.0,Jumanji (1995)
2,13,2,3.0,Jumanji (1995)
3,29,2,3.0,Jumanji (1995)
4,34,2,3.0,Jumanji (1995)


## Rating Calculation

In [ ]:
Calculate_Rating = all_in_one.dropna(axis = 0, subset = ['title'])
movie_ratingCount = (Calculate_Rating.
     groupby(by = ['title'])['rating'].
     count().
     reset_index().
     rename(columns = {'rating': 'totalRatingCount'})
     [['title', 'totalRatingCount']]
    )
movie_ratingCount.head()

,title,totalRatingCount
0,#chicagoGirl: The Social Network Takes on a Di...,2
1,$ (Dollars) (1971),11
2,$5 a Day (2008),27
3,$9.99 (2008),32
4,$ellebrity (Sellebrity) (2012),2


## Data Combination




In [ ]:
combine_all = Calculate_Rating.merge(movie_ratingCount, left_on = 'title', right_on = 'title', how = 'left')
combine_all.head()

,userId,movieId,rating,title,totalRatingCount
0,1,2,3.5,Jumanji (1995),13780
1,5,2,3.0,Jumanji (1995),13780
2,13,2,3.0,Jumanji (1995),13780
3,29,2,3.0,Jumanji (1995),13780
4,34,2,3.0,Jumanji (1995),13780


## Statistic

In [ ]:
combine_all.totalRatingCount.describe()

count    1.241928e+07
mean     8.359348e+03
std      8.738337e+03
min      1.000000e+00
25%      1.856000e+03
50%      5.294000e+03
75%      1.215000e+04
max      4.170300e+04
Name: totalRatingCount, dtype: float64

## Feltring Movies With Nombre of Rating Less Than 1000

In [ ]:
minimum_number_of_ratings = 1000
rating_popular_movie= combine_all.query('totalRatingCount >= @minimum_number_of_ratings')
rating_popular_movie.head()

,userId,movieId,rating,title,totalRatingCount
0,1,2,3.5,Jumanji (1995),13780
1,5,2,3.0,Jumanji (1995),13780
2,13,2,3.0,Jumanji (1995),13780
3,29,2,3.0,Jumanji (1995),13780
4,34,2,3.0,Jumanji (1995),13780


In [ ]:
print(rating_popular_movie.shape)

(10386435, 5)


## Creation Pivot Creation

In [ ]:
matrix_pivot =rating_popular_movie.pivot_table(index='userId',columns='title',values='rating').fillna(0)

In [ ]:
matrix_pivot

title,"'burbs, The (1989)",(500) Days of Summer (2009),*batteries not included (1987),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),12 Angry Men (1957),127 Hours (2010),13 Going on 30 (2004),...,Young Guns II (1990),Young Sherlock Holmes (1985),Zack and Miri Make a Porno (2008),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85784,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
85785,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
85786,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Sparse Matrix

In [ ]:
movie_features_df_matrix = csr_matrix(matrix_pivot.values)


In [ ]:
movie_features_df_matrix

<85788x2341 sparse matrix of type '<class 'numpy.float32'>'
	with 10386254 stored elements in Compressed Sparse Row format>

## Nearest Neighbors Algorithme (KNN)


In [ ]:
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(movie_features_df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [ ]:
movie_features_df_matrix.shape

(85788, 2341)

## Random Index

In [ ]:
query_index = np.random.choice(matrix_pivot.shape[0])
print(query_index)

18758


## Nearest Neighbors Calculation

In [ ]:
distances, indices = model_knn.kneighbors(matrix_pivot.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 7)

## Users Similar With The Random User

In [ ]:
user_ids = []
for index in range(0, len(distances.flatten())):
    user_ids.append(matrix_pivot.index[indices.flatten()[index]])
    if index == 0: # the movie chosen
        print(f"Users similar with user having user_id:  {matrix_pivot.index[query_index]}")
        print("---------------------------------------------------------")
    else:
        print(f"{index}: {matrix_pivot.index[indices.flatten()[index]]} (dist: {distances.flatten()[index]})")

Users similar with user having user_id:  18759
---------------------------------------------------------
1: 33473 (dist: 0.38116455078125)
2: 47156 (dist: 0.3889783024787903)
3: 45541 (dist: 0.3908384442329407)
4: 83516 (dist: 0.39121925830841064)
5: 29293 (dist: 0.4029301404953003)
6: 64189 (dist: 0.4084852337837219)


In [ ]:
print(user_ids)

[18759, 33473, 47156, 45541, 83516, 29293, 64189]


## Movies Recommendation

In [ ]:
# select movies that were highly ranked by the most similar users.

# look only for movies highly rated by the similar users, not the current user
candidate_user_ids = user_ids[1:]
sel_ratings = rating_popular_movie.loc[rating_popular_movie.userId.isin(candidate_user_ids)]
# sort by best ratings and total rating count
sel_ratings = sel_ratings.sort_values(by=["rating", "totalRatingCount"], ascending=False)
# eliminate from the selection movies that were ranked already by the current user
movies_rated_by_targeted_user = list(rating_popular_movie.loc[rating_popular_movie.userId==user_ids[0]]["movieId"].values)
sel_ratings = sel_ratings.loc[~sel_ratings.movieId.isin(movies_rated_by_targeted_user)]
# aggregate and count total ratings and total totalRatingCount
agg_sel_ratings = sel_ratings.groupby(["title", "rating"])["totalRatingCount"].max().reset_index()
agg_sel_ratings.columns = ["title", "rating", "total_ratings"]
agg_sel_ratings = agg_sel_ratings.sort_values(by=["rating", "total_ratings"], ascending=False)
# only select top n (default top 10 here)
rec_list = agg_sel_ratings["title"].head(10).values
print(f"\nMovies recommended to user: {indices[0][0]}\n---------------------------------")
for i, rec in enumerate(rec_list):
    print(f"{i+1}: {rec}")


Movies recommended to user: 18758
---------------------------------
1: Shawshank Redemption, The (1994)
2: American Beauty (1999)
3: Dances with Wolves (1990)
4: Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)
5: Shrek (2001)
6: Apocalypse Now (1979)
7: When Harry Met Sally... (1989)
8: Monty Python's Life of Brian (1979)
9: Amadeus (1984)
10: Life Is Beautiful (La Vita è bella) (1997)


## Fonction For Recommend Similar Users And Movies For Random User

In [ ]:
def recommend(userId=10, n_users=5, rec_top_n=10):

    distances, indices = model_knn.kneighbors(matrix_pivot.loc[matrix_pivot.index==userId].values.reshape(1, -1), n_neighbors = n_users + 1)
    user_ids = []
    for index in range(0, len(distances.flatten())):
        user_ids.append(matrix_pivot.index[indices.flatten()[index]])
        if index == 0: # the movie chosen
            print(f"Users similar with user having user_id: {userId}")
            print("---------------------------------------------------------")
        else:
            print(f"{index}: {matrix_pivot.index[indices.flatten()[index]]} (dist: {distances.flatten()[index]})")

    # select movies that were highly ranked by the most similar users.

    # look only for movies highly rated by the similar users, not the current user
    candidate_user_ids = user_ids[1:]
    sel_ratings = rating_popular_movie.loc[rating_popular_movie.userId.isin(candidate_user_ids)]
    # sort by best ratings and total rating count
    sel_ratings = sel_ratings.sort_values(by=["rating", "totalRatingCount"], ascending=False)
    # eliminate from the selection movies that were ranked already by the current user
    movies_rated_by_targeted_user = list(rating_popular_movie.loc[rating_popular_movie.userId==user_ids[0]]["movieId"].values)
    sel_ratings = sel_ratings.loc[~sel_ratings.movieId.isin(movies_rated_by_targeted_user)]
    # aggregate and count total ratings and total totalRatingCount
    agg_sel_ratings = sel_ratings.groupby(["title", "rating"])["totalRatingCount"].max().reset_index()
    agg_sel_ratings.columns = ["title", "rating", "total_ratings"]
    agg_sel_ratings = agg_sel_ratings.sort_values(by=["rating", "total_ratings"], ascending=False)
    # only select top n (default top 10 here)
    rec_list = agg_sel_ratings["title"].head(10).values
    print(f"\nMovies recommended to user_id: {user_ids[0]}\n---------------------------------")
    for i, rec in enumerate(rec_list):
        print(f"{i+1}: {rec}")

In [ ]:
recommend(10, 5, 10)

Users similar with user having user_id: 10
---------------------------------------------------------
1: 55 (dist: 0.5052522420883179)
2: 10420 (dist: 0.5308452844619751)
3: 41246 (dist: 0.5530043244361877)
4: 37415 (dist: 0.5623594522476196)
5: 54649 (dist: 0.5722603797912598)

Movies recommended to user_id: 10
---------------------------------
1: Terminator 2: Judgment Day (1991)
2: One Flew Over the Cuckoo's Nest (1975)
3: Aliens (1986)
4: Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)
5: Platoon (1986)
6: Rocky (1976)
7: Boot, Das (Boat, The) (1981)
8: Goldfinger (1964)
9: Great Escape, The (1963)
10: Deer Hunter, The (1978)
